In [ ]:

import os
import pandas as pd


In [55]:
import pandas as pd
import numpy as np

class MatrixLoader:
    def __init__(self, year, agg):
        self.base_url = 'https://github.com/GuilhermeZiegler/input_output/raw/master/input_output/data/iom'
        self.year = year
        self.agg = agg
        self.file_path = f'{self.base_url}/{self.agg}/{self.year}/MIP-BR-CN10-68S-{self.year}.xlsx'
        self.sheet_data = {}  # Dictionary to hold data from each sheet
        self.load_data()

    def load_data(self):
        xls = pd.ExcelFile(self.file_path)
        for sheet_name in xls.sheet_names:
            if sheet_name == 'Referência':
                continue  # Ignore the 'Referência' sheet
            self.extract_indices_and_matrix(sheet_name, xls)

    def extract_indices_and_matrix(self, sheet_name, xls):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        
        # Extract row indexes starting from A5
        row_start_index = 4
        row_indexes = df.iloc[row_start_index:, :3].reset_index(drop=True)

        # Extract column indexes starting from D2
        col_start_row, col_end_row = 1, 3
        col_start_col = 3
        column_indexes = df.iloc[col_start_row:col_end_row+1, col_start_col:].reset_index(drop=True)

        # Extract data matrix starting from D5
        data_matrix = df.iloc[row_start_index:, col_start_col:].reset_index(drop=True)
        data_matrix = data_matrix.dropna(axis=1, how='all')

        # Store data in the dictionary
        self.sheet_data[sheet_name] = {
            'row_indexes': row_indexes,
            'column_indexes': column_indexes,
            'data_matrix': data_matrix,
            'aligned_matrix': self.align_matrix(row_indexes, column_indexes, data_matrix)
        }

    def align_matrix(self, row_indexes, column_indexes, data_matrix):
        # Ensure the data matrix and column indexes have matching shapes
        col_len_data = data_matrix.shape[1]
        col_len_index = column_indexes.shape[1]

        if col_len_data < col_len_index:
            # Add NaN columns to data matrix
            for _ in range(col_len_index - col_len_data):
                data_matrix[f'NaN_{_}'] = np.nan
        elif col_len_data > col_len_index:
            # Truncate the data matrix columns
            data_matrix = data_matrix.iloc[:, :col_len_index]

        # Create MultiIndex for rows
        row_index = pd.MultiIndex.from_frame(row_indexes)

        # Create MultiIndex for columns
        col_index = pd.MultiIndex.from_frame(column_indexes.T)

        # Align the data matrix with NaN filling for missing values
        aligned_df = pd.DataFrame(data_matrix.values, index=row_index, columns=col_index)
        aligned_df = aligned_df.reindex(index=row_index, columns=col_index)

        return aligned_df

    def get_aligned_matrix(self, sheet_name):
        """Return the aligned matrix for a specific sheet."""
        return self.sheet_data.get(sheet_name, {}).get('aligned_matrix', None)

    def display_aligned_matrices(self):
        """Display the aligned matrices for all sheets."""
        for sheet_name, data in self.sheet_data.items():
            print(f"Planilha: {sheet_name}")
            print("Matriz Alinhada:")
            display(data['aligned_matrix'])
            print("\n")

# Exemplo de uso
year = '2010'
agg = '68S'
loader = MatrixLoader(year, agg)

# Exibir as matrizes alinhadas para todas as planilhas
loader.display_aligned_matrices()


Planilha: Producao
Matriz Alinhada:


0                                                                                    01911  \
1                                                            Arroz, trigo e outros cereais   
2                                                                                    1.0     
0    1                                                  2                                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                           7557   
0192 Pecuária, inclusive o apoio à pecuária             2.0                            280   
0280 Produção florestal; pesca e aquicultura            3.0                             17   
0580 Extração de carvão mineral e de minerais não-me... 4.0                              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                              0   
...                                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0                             0   
9700 Serviços domésticos                                68.0                             0   
NaN  Total\ndo produto                                  69.0                          8042   
     NaN                                                NaN                            NaN   
     Conferência                                        NaN                              0   

0                                                                    01912  \
1                                                            Milho em grão   
2                                                                    2.0     
0    1                                                  2                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0          13554   
0192 Pecuária, inclusive o apoio à pecuária             2.0           1000   
0280 Produção florestal; pesca e aquicultura            3.0             30   
0580 Extração de carvão mineral e de minerais não-me... 4.0              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0              0   
...                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0             0   
9700 Serviços domésticos                                68.0             0   
NaN  Total\ndo produto                                  69.0         14717   
     NaN                                                NaN            NaN   
     Conferência                                        NaN              0   

0                                                                                                         01913  \
1                                                            Algodão herbáceo, outras fibras da lav. temporária   
2                                                                                                         3.0     
0    1                                                  2                                                         
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                                                5155   
0192 Pecuária, inclusive o apoio à pecuária             2.0                                                   6   
0280 Produção florestal; pesca e aquicultura            3.0                                                   0   
0580 Extração de carvão mineral e de minerais não-me... 4.0                                                   0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                                                   0   
...                                                                                                         ...   
9480 Organizações associativas e outros serviços pes... 67.0                                                  0   
9700 Serviços domésticos                                68.0                                                  0   
NaN  Total\ndo produto                



Planilha: Usos PxS
Matriz Alinhada:


0                                                                                                                     0191  \
1                                                            Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                     1.0    
0     1                                                  2                                                                   
01911 Arroz, trigo e outros cereais                      1.0                                         133.429818              
01912 Milho em grão                                      2.0                                         470.059704              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                          96.352281              
01914 Cana-de-açúcar                                     4.0                                           546.4397              
01915 Soja  em grão                                      5.0                                         860.895961              
...                                                                                                         ...              
NaN   NaN                                                NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              

0                                                                                              0192  \
1                                                            Pecuária, inclusive o apoio à pecuária   
2                                                                                              2.0    
0     1                                                  2                                            
01911 Arroz, trigo e outros cereais                      1.0                             136.408162   
01912 Milho em grão                                      2.0                             1818.21705   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                              13.642801   
01914 Cana-de-açúcar                                     4.0                              75.147056   
01915 Soja  em grão                                      5.0                              73.664294   
...                                                                                             ...   
NaN   NaN                                                NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   

0                                                                                               0280  \
1                                                            Produção florestal; pesca e aquicultura   
2                                                                                               3.0    
0     1                                                  2                                             
01911 Arroz, trigo e outros cereais                      1.0                                       0   
01912 Milho em grão                        



Planilha: Usos SxS
Matriz Alinhada:


...

0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                            -7.353098              
01912 Milho em grão                                      2.0                                           -13.335939              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             2.338972              
01914 Cana-de-açúcar                                     4.0                                            14.901089              
01915 Soja  em grão                                      5.0                                             6.796178              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                        2253.695044              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                 -7.51723   
01912 Milho em grão                                      2.0                               -51.584154   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 0.331182   
01914 Cana-de-açúcar                                     4.0                                 2.049216   
01915 Soja  em grão                                      5.0                                 0.581529   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                            1029.266649   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   



Planilha: MG Com
Matriz Alinhada:


0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                             22.08087              
01912 Milho em grão                                      2.0                                            67.258588              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             7.435465              
01914 Cana-de-açúcar                                     4.0                                                    0              
01915 Soja  em grão                                      5.0                                            58.632968              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                         6427.35874              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                22.573746   
01912 Milho em grão                                      2.0                               260.159954   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 1.052809   
01914 Cana-de-açúcar                                     4.0                                        0   
01915 Soja  em grão                                      5.0                                 5.017048   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                            4794.343732   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   



Planilha: MG Transp
Matriz Alinhada:


0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                            12.802108              
01912 Milho em grão                                      2.0                                            42.015693              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             3.050447              
01914 Cana-de-açúcar                                     4.0                                            27.094399              
01915 Soja  em grão                                      5.0                                            36.384446              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                         813.329959              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                 13.08787   
01912 Milho em grão                                      2.0                               162.519035   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 0.431922   
01914 Cana-de-açúcar                                     4.0                                 3.726055   
01915 Soja  em grão                                      5.0                                 3.113308   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                             753.622067   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   

In [56]:
# Exibir as matrizes alinhadas para todas as planilhas
loader.display_aligned_matrices()

Planilha: Producao
Matriz Alinhada:


0                                                                                    01911  \
1                                                            Arroz, trigo e outros cereais   
2                                                                                    1.0     
0    1                                                  2                                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                           7557   
0192 Pecuária, inclusive o apoio à pecuária             2.0                            280   
0280 Produção florestal; pesca e aquicultura            3.0                             17   
0580 Extração de carvão mineral e de minerais não-me... 4.0                              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                              0   
...                                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0                             0   
9700 Serviços domésticos                                68.0                             0   
NaN  Total\ndo produto                                  69.0                          8042   
     NaN                                                NaN                            NaN   
     Conferência                                        NaN                              0   

0                                                                    01912  \
1                                                            Milho em grão   
2                                                                    2.0     
0    1                                                  2                    
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0          13554   
0192 Pecuária, inclusive o apoio à pecuária             2.0           1000   
0280 Produção florestal; pesca e aquicultura            3.0             30   
0580 Extração de carvão mineral e de minerais não-me... 4.0              0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0              0   
...                                                                    ...   
9480 Organizações associativas e outros serviços pes... 67.0             0   
9700 Serviços domésticos                                68.0             0   
NaN  Total\ndo produto                                  69.0         14717   
     NaN                                                NaN            NaN   
     Conferência                                        NaN              0   

0                                                                                                         01913  \
1                                                            Algodão herbáceo, outras fibras da lav. temporária   
2                                                                                                         3.0     
0    1                                                  2                                                         
0191 Agricultura, inclusive o apoio à agricultura e ... 1.0                                                5155   
0192 Pecuária, inclusive o apoio à pecuária             2.0                                                   6   
0280 Produção florestal; pesca e aquicultura            3.0                                                   0   
0580 Extração de carvão mineral e de minerais não-me... 4.0                                                   0   
0680 Extração de petróleo e gás, inclusive as ativid... 5.0                                                   0   
...                                                                                                         ...   
9480 Organizações associativas e outros serviços pes... 67.0                                                  0   
9700 Serviços domésticos                                68.0                                                  0   
NaN  Total\ndo produto                



Planilha: Usos PxS
Matriz Alinhada:


0                                                                                                                     0191  \
1                                                            Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                     1.0    
0     1                                                  2                                                                   
01911 Arroz, trigo e outros cereais                      1.0                                         133.429818              
01912 Milho em grão                                      2.0                                         470.059704              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                          96.352281              
01914 Cana-de-açúcar                                     4.0                                           546.4397              
01915 Soja  em grão                                      5.0                                         860.895961              
...                                                                                                         ...              
NaN   NaN                                                NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              
                                                         NaN                                                  0              

0                                                                                              0192  \
1                                                            Pecuária, inclusive o apoio à pecuária   
2                                                                                              2.0    
0     1                                                  2                                            
01911 Arroz, trigo e outros cereais                      1.0                             136.408162   
01912 Milho em grão                                      2.0                             1818.21705   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                              13.642801   
01914 Cana-de-açúcar                                     4.0                              75.147056   
01915 Soja  em grão                                      5.0                              73.664294   
...                                                                                             ...   
NaN   NaN                                                NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   
                                                         NaN                                      0   

0                                                                                               0280  \
1                                                            Produção florestal; pesca e aquicultura   
2                                                                                               3.0    
0     1                                                  2                                             
01911 Arroz, trigo e outros cereais                      1.0                                       0   
01912 Milho em grão                        



Planilha: Usos SxS
Matriz Alinhada:


...

0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                            -7.353098              
01912 Milho em grão                                      2.0                                           -13.335939              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             2.338972              
01914 Cana-de-açúcar                                     4.0                                            14.901089              
01915 Soja  em grão                                      5.0                                             6.796178              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                        2253.695044              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                 -7.51723   
01912 Milho em grão                                      2.0                               -51.584154   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 0.331182   
01914 Cana-de-açúcar                                     4.0                                 2.049216   
01915 Soja  em grão                                      5.0                                 0.581529   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                            1029.266649   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   



Planilha: MG Com
Matriz Alinhada:


0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                             22.08087              
01912 Milho em grão                                      2.0                                            67.258588              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             7.435465              
01914 Cana-de-açúcar                                     4.0                                                    0              
01915 Soja  em grão                                      5.0                                            58.632968              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                         6427.35874              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                22.573746   
01912 Milho em grão                                      2.0                               260.159954   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 1.052809   
01914 Cana-de-açúcar                                     4.0                                        0   
01915 Soja  em grão                                      5.0                                 5.017048   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                            4794.343732   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   



Planilha: MG Transp
Matriz Alinhada:


0                                                                                                                       0191  \
1                                                              Agricultura, inclusive o apoio à agricultura e a pós-colheita   
2                                                                                                                       1.0    
0     1                                                  2                                                                     
01911 Arroz, trigo e outros cereais                      1.0                                            12.802108              
01912 Milho em grão                                      2.0                                            42.015693              
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                             3.050447              
01914 Cana-de-açúcar                                     4.0                                            27.094399              
01915 Soja  em grão                                      5.0                                            36.384446              
...                                                                                                           ...              
94801 Organizações patronais, sindicais e outros serv... 125.0                                                  0              
94802 Manutenção de computadores, telefones e objetos... 126.0                                                  0              
94803 Serviços pessoais                                  127.0                                                  0              
97001 Serviços domésticos                                128.0                                                  0              
NaN   Prod Nac                                           129.0                                         813.329959              

0                                                                                                0192  \
1                                                              Pecuária, inclusive o apoio à pecuária   
2                                                                                                2.0    
0     1                                                  2                                              
01911 Arroz, trigo e outros cereais                      1.0                                 13.08787   
01912 Milho em grão                                      2.0                               162.519035   
01913 Algodão herbáceo, outras fibras da lav. temporária 3.0                                 0.431922   
01914 Cana-de-açúcar                                     4.0                                 3.726055   
01915 Soja  em grão                                      5.0                                 3.113308   
...                                                                                               ...   
94801 Organizações patronais, sindicais e outros serv... 125.0                                      0   
94802 Manutenção de computadores, telefones e objetos... 126.0                                      0   
94803 Serviços pessoais                                  127.0                                      0   
97001 Serviços domésticos                                128.0                                      0   
NaN   Prod Nac                                           129.0                             753.622067   

0                                                                                                 0280  \
1                                                              Produção florestal; pesca e aquicultura   
2                                                                                                 3.0    
0     1                                                  2                                               
01911 Arroz, trigo e outros cereais                      1.0                   